## Builder‘s Guide

### Layers and Modules

- Mulitple layers are combined into modules

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
torch.linspace(-1, 1, steps=4)

tensor([-1.0000, -0.3333,  0.3333,  1.0000])

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand((2, 10))
net(X).shape

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\automl\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

#### A Custom Module

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
net = MLP()
net(X).shape

torch.Size([2, 10])

In [5]:
net

MLP(
  (hidden): Linear(in_features=10, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
len(net.state_dict())

4

In [7]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0358,  0.2286, -0.0150,  ..., -0.1135, -0.1324, -0.2940],
                      [-0.2464, -0.3121, -0.1574,  ..., -0.1528, -0.2828, -0.2669],
                      [-0.0987,  0.3077,  0.0733,  ..., -0.0557, -0.0490,  0.1157],
                      ...,
                      [ 0.0028, -0.0786, -0.1766,  ..., -0.2333,  0.2971,  0.2376],
                      [ 0.2514,  0.1688,  0.2754,  ...,  0.0757, -0.0635, -0.2837],
                      [ 0.1652,  0.2226,  0.1951,  ...,  0.2560,  0.1896,  0.2870]])),
             ('hidden.bias',
              tensor([-0.2673, -0.2728, -0.1025, -0.2329,  0.1333, -0.0797,  0.0279,  0.1653,
                      -0.0537,  0.0195, -0.2759,  0.0578, -0.1973,  0.2410,  0.1599,  0.0004,
                       0.1304, -0.0534,  0.3011,  0.3027, -0.1651,  0.2628,  0.2966, -0.2835,
                      -0.1548, -0.1457,  0.1050,  0.2067,  0.0460,  0.1584,  0.0276, -0.0597,
                      -0.2992,

#### The Sequential Module

- A method to append modules one by one to a list
- A forward propagation method to pass an input through the chain of modules, in the same order as they were appended

In [8]:
# using "childrne" class to access added modules

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, moduels in enumerate(args):
            self.add_module('l' + str(idx), moduels)
    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In [9]:
net = MySequential(nn.LazyLinear(256),nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [10]:
net

MySequential(
  (l0): Linear(in_features=10, out_features=256, bias=True)
  (l1): Linear(in_features=256, out_features=256, bias=True)
  (l2): ReLU()
  (l3): Linear(in_features=256, out_features=10, bias=True)
)

In [11]:
net.l0

Linear(in_features=10, out_features=256, bias=True)

#### Executing Code in the Forward Propagation Method

- Perform arbitrary matchmatical operations, not simpely relying on predifned NNs
- sharing one linear layer and add some mathmatical operaitons

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [13]:
X = torch.arange(40.0).reshape(2, 20)
net = FixedHiddenMLP()
net(X)

tensor(-0.0563, grad_fn=<SumBackward0>)

In [14]:
# mixing various modules together
class NestMlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(20), nn.ReLU(),
                                nn.LazyLinear(50), nn.ReLU())
        self.linear = nn.LazyLinear(60)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMlp(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.1169, grad_fn=<SumBackward0>)

In [15]:
chimera

Sequential(
  (0): NestMlp(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
      (2): Linear(in_features=20, out_features=50, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=50, out_features=60, bias=True)
  )
  (1): Linear(in_features=60, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)

#### Summary

- Layers are modules. Many layers can comprise a module. Many modules can comprise a module.  
- A module can contain code. Modules take care of lots of housekeeping, including parameter initialization and backpropagation. Sequential concatenations of layers and modules are handled by the Sequential module

#### Exerises

1. What kinds of problems will occur if you change MySequential to store modules in a Python list?

In [16]:
class MySequentialExe(nn.Module):
    def __init__(self, *args):
        super().__init__()

        self.layers = []
        for idx, layer in enumerate(args): # convert module inputs to list
            self.layers.append(layer)

    def forward(self, X):
        for block in self.layers:
            X = block(X)
        return X

x = torch.rand(size=(2, 10))
net = MySequentialExe(nn.LazyLinear(8), nn.LazyLinear(4), nn.LazyLinear(1))
net

MySequentialExe()

In [17]:
net(x)

tensor([[0.5252],
        [0.5746]], grad_fn=<AddmmBackward0>)

In [18]:
net.state_dict()

OrderedDict()

2. Implement a module that takes two modules as an argument, say net1 and net2 and returns the concatenated output of both networks in the forward propagation. This is also called a **parallel module**.

3. Assume that you want to concatenate multiple instances of the same network. Implement a factory function that generates multiple instances of the same module and build a larger network from it.

### Parameter Management

- Accessing parameters for debugging, diagnostic, and visualizations
- Sharing parameters across different model components.

In [19]:
import torch
from torch import nn

In [20]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0726],
        [0.1088]], grad_fn=<AddmmBackward0>)

In [21]:
2.tolist()

SyntaxError: invalid syntax (696132364.py, line 1)

#### Parameter access

In [ ]:
net[2].state_dict()

In [ ]:
type(net[2].bias), net[2].bias.data

In [ ]:
# we have not invoked backprogation for this NN
net[2].weight.grad == None

In [ ]:
# all parameters
[(name, param.shape) for name, param in net.named_parameters()]

#### Tied parameters

In [ ]:
# we need to give a shared layer a name so that we can refer to
# its  parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)

In [ ]:
net

In [ ]:
net[2].weight.data == net[4].weight.data

In [ ]:
# modify one of the parameters and keep still
net[2].weight.data[0, 0] = 100
net[2].weight.data == net[4].weight.data